In [1]:
import pandas as pd
import sqlite3 as sql
import numpy as np
import re
import string

In [97]:
## join epa vehicle information to adjusted weight information for 2023-2017 years.
epa_df = pd.read_xml("/Users/josheverts/Downloads/vehicles.xml")

## data to join with adjusted weights
test_data_2023 = pd.read_excel('/Users/josheverts/Downloads/23-testcar-2022-11-03.xlsx')
test_data_2022 = pd.read_excel('/Users/josheverts/Downloads/22-testcar-2023-02-28.xlsx')
test_data_2021 = pd.read_excel('/Users/josheverts/Downloads/21-tstcar-2022-04-15.xlsx')
test_data_2020 = pd.read_excel('/Users/josheverts/Downloads/20tstcar-2021-03-02.xlsx')
test_data_2019 = pd.read_excel('/Users/josheverts/Downloads/19tstcar-2020-10-02.xlsx')
test_data_2018 = pd.read_excel('/Users/josheverts/Downloads/18tstcar-2018-10-24.xlsx')
test_data_2017 = pd.read_excel('/Users/josheverts/Downloads/17tstcar-2018-05-30.xlsx')

In [98]:
weight_dfs = [test_data_2023, test_data_2022, test_data_2021,
              test_data_2020, test_data_2019, test_data_2018, test_data_2017]

In [99]:
def clean_index_adj_weights(adj_weight_df):
    adj_weight_df_grouped = adj_weight_df.groupby(['Represented Test Veh Make',
                                                   'Represented Test Veh Model', 
                                                   'Veh Mfr Code'], group_keys = False).mean(numeric_only = True)
    adj_weight_df_grouped = adj_weight_df_grouped.reset_index()
    adj_weight_df_grouped = adj_weight_df_grouped.reset_index()
    adj_weight_df_grouped = adj_weight_df_grouped.rename(columns = {'index':'VehicleID'})
    rename_dict = {'Model Year': 'Year', 'Represented Test Veh Make':'Make', 'Veh Mfr Code': 'MFRCode',
               'Represented Test Veh Model':'Model', 'Equivalent Test Weight (lbs.)':'AdjWeight'}
    epa_adj_weights = adj_weight_df_grouped.rename(rename_dict, axis=1)  
    epa_adj_weights = epa_adj_weights[['VehicleID', 'Year', 'MFRCode', 'Make', 'Model', 'AdjWeight']]
    epa_adj_weights.astype({'Year': 'int32'})
    
    return epa_adj_weights

def clean_index_vehicle_data(vehicle_df):
    
    ## rename columns of each df
    rename_dict = {'id': 'VehicleID', 'make': 'Make', 'model':'Model', 'mfrCode':'MFRCode', 'range': 'Range',
              'rangeHwy': 'RangeHwy', 'year':'Year'}
    vehicle_df = vehicle_df.rename(rename_dict, axis=1)
    
    ## vehicle atvType are:
    vehicle_df['atvType'].unique()
    ## if None or Diesel, assign ICE
    ## if Hybrid assign HEV
    ## if Plug-in Hybrid assigh PHEV
    ## if FFV assign FCV
    def convert_vals(vals):
        out = []
        for val in vals:
            if pd.isna(val) == True or val == 'Diesel':
                out.append('ICE')
            elif val == 'Hybrid':
                out.append('HEV')
            elif val == 'Plug-in Hybrid':
                out.append('PHEV')
            elif val == 'EV':
                out.append(val)
            elif val == 'FFV':
                out.append('FCV')
            else:
                out.append(None)
        return out
    new_codes = convert_vals(vehicle_df['atvType'])
    vehicle_df['atvType'] = new_codes
    vehicle_df = vehicle_df[['VehicleID', 'Year', 'MFRCode', 'Make', 'Model', 
                             'Range', 'RangeHwy', 'rangeHwyA','atvType', 
                             'UHighway', 'UCity', 'city08U', 'highway08U', 'combE', 
                             'combinedUF', 'comb08', 'trany', 'cylinders', 'displ', 'baseModel']]
    
    return vehicle_df

def concat_weight_data(weight_dfs):
    combined = pd.DataFrame()
    for df in weight_dfs:
        cleaned = clean_index_adj_weights(df)
        combined = pd.concat([combined, cleaned])
    combined['Year'] = combined['Year'].astype('int')
    combined['VehicleID'] = np.arange(0, len(combined)) ## assign new unique ids
    combined = combined.reset_index().drop(['index'], axis = 1)
    return combined
        
    
def vehicle_data_join(vehicle_df, weight_dfs):
    combined_weights = concat_weight_data(weight_dfs)
    tempdf = vehicle_df[['VehicleID','Year','Make', 'Model']]
    tempdf2 = vehicle_df[['VehicleID', 'Make', 'Model', 'trany', 'displ', 'cylinders', 'atvType']]
    vehicle_df['Make'] = vehicle_df['Make'].str.lower()
    vehicle_df['baseModel'] = vehicle_df['baseModel'].str.lower()
    vehicle_df['Model'] = vehicle_df['Model'].str.lower()
    combined_weights['Make'] = combined_weights['Make'].str.lower()
    combined_weights['Model'] = combined_weights['Model'].str.lower()
    combined_weights['baseModel'] = [i.split(' ')[0] for i in combined_weights['Model']]
    adj_join = pd.merge(epa_df, combined_weights, how='inner', 
                        left_on=['Year', 'Make','baseModel'], right_on = ['Year', 'Make','baseModel'])
    adj_join_g = adj_join.groupby(['VehicleID_x']).agg({'UHighway': np.mean, 'UCity': np.mean, 'city08U': np.mean, 
                                                    'Range': np.mean, 'RangeHwy': np.mean, 'rangeHwyA': np.mean,
                                                    'highway08U':  np.mean, 'combE': np.mean, 'combinedUF': np.mean, 
                                                    'comb08': np.mean, 'AdjWeight': np.mean})  
    adj_join_g_j = pd.merge(adj_join_g, tempdf2, how = 'inner', left_on='VehicleID_x', right_on='VehicleID')
    adj_join_final = adj_join_g_j[~adj_join_g_j.duplicated(['Make', 'Model', 'trany', 'displ', 'cylinders'])]
    adj_join_final_cap = pd.merge(adj_join_final, tempdf, how = 'inner', left_on='VehicleID', right_on='VehicleID')
    adj_join_final_cap_drop = adj_join_final_cap.drop(['Make_x', 'Model_x'], axis = 1)
    adj_join_final_cap_drop = adj_join_final_cap_drop.rename({'Make_y': 'Make', 'Model_y': 'Model'}, axis = 1)
    cols = adj_join_final_cap_drop.columns.to_list()
    cols = cols[::-1]
    adj_join_final_cap_drop = adj_join_final_cap_drop[cols]
    
    return adj_join_final_cap_drop

    


# # select 2023 years from epa range/efficiency data
# vehicle_df = vehicle_df[vehicle_df['year'] == 2023]

# ## rename columns of each df
# rename_dict = {'id': 'VehicleID', 'make': 'Make', 'model':'Model', 'mfrCode':'MFRCode', 'range': 'Range',
#               'rangeHwy': 'RangeHwy', 'year':'Year'}
# vehicle_df = vehicle_df.rename(rename_dict, axis=1)  
# rename_dict = {'Model Year': 'Year', 'Represented Test Vehicle Model':'Model', 
#                'Represented Test Vehicle Make': 'Make', 'Curb Weight (lbs.)':'CurbWeight', 'Displacement (L)' : 'displ'}
# epa_curb_weights = epa_curb_weights.rename(rename_dict, axis=1)  
# rename_dict = {'Model Year': 'Year', 'Represented Test Veh Make':'Make', 'Veh Mfr Code': 'MFRCode',
#                'Represented Test Veh Model':'Model', 'Equivalent Test Weight (lbs.)':'AdjWeight'}
# epa_adj_weights = adj_weight_df_grouped.rename(rename_dict, axis=1)  

# ## take selected columns from each df
# vehicle_df = vehicle_df[['VehicleID', 'Year', 'MFRCode', 'Make', 'Model', 'Range', 'RangeHwy', 'rangeHwyA','atvType', 
#                  'UHighway', 'UCity', 'city08U', 'highway08U', 'combE', 
#                  'combinedUF', 'comb08', 'trany', 'cylinders', 'displ', 'baseModel']]
# epa_curb_weights = epa_curb_weights[['VehicleID', 'Year', 'Make', 'Model', 'CurbWeight', 'displ']]
# epa_adj_weights = epa_adj_weights[['VehicleID', 'Year', 'MFRCode', 'Make', 'Model', 'AdjWeight']]

# ## convert years back to int
# epa_curb_weights.astype({'Year': 'int32'})
# epa_adj_weights.astype({'Year': 'int32'})



In [100]:
epa_df = clean_index_vehicle_data(epa_df)

In [101]:
joined_data = vehicle_data_join(epa_df, weight_dfs)

In [102]:
joined_data

,Model,Make,Year,atvType,cylinders,displ,trany,VehicleID,AdjWeight,comb08,combinedUF,combE,highway08U,rangeHwyA,RangeHwy,Range,city08U,UCity,UHighway
0,Q7,Audi,2017,ICE,6.0,3.0,Automatic (S8),37221,5062.500000,21.0,0.0,0.0000,25.0209,0.0,0.0,0.0,18.7066,23.5000,35.2000
1,Elantra SE,Hyundai,2017,ICE,4.0,2.0,Automatic (S6),37225,3198.660714,33.0,0.0,0.0000,38.3294,0.0,0.0,0.0,29.0775,38.2873,56.5318
2,Elantra,Hyundai,2017,ICE,4.0,2.0,Automatic (S6),37226,3198.660714,32.0,0.0,0.0000,36.8668,0.0,0.0,0.0,28.1626,36.9258,54.0869
3,Elantra,Hyundai,2017,ICE,4.0,2.0,Manual 6-spd,37227,3198.660714,29.0,0.0,0.0000,35.6666,0.0,0.0,0.0,25.7569,33.4000,52.1000
4,Sportage FWD,Kia,2017,ICE,4.0,2.4,Automatic (S6),37262,3840.909091,25.0,0.0,0.0000,28.9611,0.0,0.0,0.0,22.3990,28.6064,41.3075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2055,AMG EQS 4matic Plus,Mercedes-Benz,2022,EV,NaN,NaN,Automatic (A1),46330,6000.000000,77.0,0.0,43.9782,77.7700,0.0,281.3,277.0,75.7400,108.2000,111.1000
2056,EQB 300 4matic,Mercedes-Benz,2022,EV,NaN,NaN,Automatic (A1),46331,5000.000000,101.0,0.0,33.3166,98.4515,0.0,236.0,243.0,103.5003,139.3620,132.6042
2057,EQB 350 4matic,Mercedes-Benz,2022,EV,NaN,NaN,Automatic (A1),46332,5000.000000,96.0,0.0,35.0000,93.0000,0.0,220.8,227.0,98.0000,135.3724,128.7000
2058,Bronco Sport 4WD,Ford,2023,ICE,3.0,1.5,Automatic 8-spd,46385,5665.178571,26.0,0.0,0.0000,28.0000,0.0,0.0,0.0,25.3012,32.7408,43.3816


In [103]:
joined_data.to_csv("adj_weight_data_join_2017-2023.csv", index = False)